In [49]:
from glob import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import cv2
import keras
import random


batch_size = 32
num_classes = 4
epochs = 10
train_percent = .8
imgScale = .3

# input image dimensions
img_rows, img_cols = 256, 1600

# set paths to train and test image datasets
TRAIN_PATH = 'train_images/'
TEST_PATH = 'test_images/'

# load dataframe with train labels
train_df = pd.read_csv('train.csv')
train_df = train_df.reindex(np.random.permutation(train_df.index))


In [50]:
train_fns = sorted(glob(TRAIN_PATH + '*.jpg'))
test_fns = sorted(glob(TEST_PATH + '*.jpg'))

print('There are {} images in the train set.'.format(len(train_fns)))
print('There are {} images in the test set.'.format(len(test_fns)))

train_df.head(10)

# split column
split_df = train_df["ImageId_ClassId"].str.split("_", n = 1, expand = True)

# add new columns to train_df
train_df['Image'] = split_df[0]
train_df['Label'] = split_df[1]

# check the result
train_df.head()

defect1 = train_df[train_df['Label'] == '1'].EncodedPixels.count()
defect2 = train_df[train_df['Label'] == '2'].EncodedPixels.count()
defect3 = train_df[train_df['Label'] == '3'].EncodedPixels.count()
defect4 = train_df[train_df['Label'] == '4'].EncodedPixels.count()
min_count = min([defect1, defect2, defect3, defect4])

path = TRAIN_PATH
count_one = 0;
count_two = 0;
count_three = 0;
count_four = 0;
count_one_test = 0;
count_two_test = 0;
count_three_test = 0;
count_four_test = 0;


x_train = []
y_train = []
x_test = []
y_test =[]

train_count = int(min_count*train_percent)
test_count = int((1-train_percent)*min_count)
images = sorted(glob(path + '*.jpg'))

for im in range(0, len(images)):
    label = int(train_df.iloc[im][3])-1

    orgimage = cv2.imread(path+train_df.iloc[im][2])
    newX,newY = orgimage.shape[1]*imgScale, orgimage.shape[0]*imgScale
    image = cv2.resize(orgimage,(int(newX),int(newY)))

    if(train_df.iloc[im][1]) != 'nan':
        if(label == 0):
            if(count_one <= train_count):
                count_one+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_one_test <= test_count):
                count_one_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 1):
            if(count_two <= train_count):
                count_two+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_two_test <= test_count):
                count_two_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 2):
            if(count_three <= train_count):
                count_three+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_three_test <= test_count):
                count_three_test+=1
                x_test.append(image)
                y_test.append(label)
        if(label == 3):
            if(count_four <= train_count):
                count_four+=1
                x_train.append(image)
                y_train.append(label)
            elif(count_four_test <= test_count):
                count_four_test+=1
                x_test.append(image)
                y_test.append(label)

    #print(len(y_test), len(x_test), len(y_train), len(x_train))
    #print(count_four_test,count_three_test,count_two_test,count_one_test, test_count)
    all_data_count = test_count
    if(count_four_test >= all_data_count and
       count_three_test >= all_data_count and
       count_two_test >= all_data_count and
       count_one_test >= all_data_count):
        break;


There are 12568 images in the train set.
There are 1801 images in the test set.


In [32]:
print(y_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 3, 3, 3, 0, 0, 3, 3, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 3, 0, 0, 3, 3, 3, 3, 0, 1, 1, 3, 1, 0, 3, 1, 1, 1, 1, 3, 0, 0, 1, 1, 3, 0, 0, 1, 3, 1, 0, 1, 1, 1, 3, 3, 2, 3, 2, 3, 2, 0, 2, 3, 0, 2, 3, 2, 3, 0, 0, 3, 3, 2, 2, 0, 3, 1, 0, 2, 0, 0, 2, 3, 0, 2, 1, 0, 2, 3, 2, 1, 3, 3, 2, 2, 0, 3, 0, 1, 0, 3, 0, 2, 2, 2, 2, 0, 3, 0, 2, 3, 2, 2, 2, 2, 3, 0, 0, 2, 3, 0, 0, 0, 3, 3, 0, 0, 2, 3, 3, 3, 3, 2, 2, 3, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [51]:
y_train_orig = y_train
y_test_orig = y_test
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
y_train = keras.utils.to_categorical(y_train,4)
y_test = keras.utils.to_categorical(y_test,4)

In [52]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), strides=(1, 1),
                 activation='relu',
                 input_shape=(image.shape[0],image.shape[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2))) 
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 792 samples, validate on 199 samples
Epoch 1/10
792/792 [==============================] - 32s 40ms/step - loss: 27.0993 - accuracy: 0.6398 - val_loss: 0.6781 - val_accuracy: 0.7060
Epoch 2/10
792/792 [==============================] - 33s 42ms/step - loss: 0.6461 - accuracy: 0.7412 - val_loss: 0.6914 - val_accuracy: 0.7048
Epoch 3/10
792/792 [==============================] - 33s 42ms/step - loss: 0.5748 - accuracy: 0.7708 - val_loss: 0.7243 - val_accuracy: 0.6997
Epoch 4/10
792/792 [==============================] - 33s 41ms/step - loss: 0.4916 - accuracy: 0.8040 - val_loss: 0.7408 - val_accuracy: 0.7123
Epoch 5/10
792/792 [==============================] - 33s 41ms/step - loss: 0.3872 - accuracy: 0.8403 - val_loss: 0.8376 - val_accuracy: 0.6935
Epoch 6/10
792/792 [==============================] - 33s 42ms/step - loss: 0.3421 - accuracy: 0.8718 - val_loss: 0.9400 - val_accuracy: 0.6746
Epoch 7/10
792/792 [==============================] - 34s 43ms/step - loss: 0.4667 - accu

In [53]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x = model.predict_classes(x_test, verbose=1)
print(x)

print(y_test_orig)
print(y_test)


Test loss: 1.0026971784668353
Test accuracy: 0.697236180305481
199/199 [==============================] - 2s 8ms/step
[1 1 0 1 2 3 2 0 0 0 0 1 3 0 1 2 3 1 2 2 2 0 2 1 0 0 0 0 2 0 0 0 0 2 2 0 2
 0 0 0 1 0 0 2 2 1 0 1 2 1 1 0 2 3 0 0 0 3 0 0 2 0 0 0 0 0 2 0 2 0 0 0 3 0
 0 0 0 3 0 0 0 3 0 0 2 3 2 0 0 0 0 0 0 0 0 1 0 0 0 2 0 0 0 2 0 3 0 0 1 0 2
 0 0 0 3 1 0 0 1 0 2 0 2 0 3 1 0 1 1 2 0 0 1 0 3 0 0 1 2 1 0 2 0 3 0 0 0 0
 0 0 2 0 0 1 0 3 0 0 1 0 0 3 2 2 0 0 2 0 0 1 2 2 0 0 0 2 0 1 0 0 0 3 0 0 0
 0 3 1 0 3 0 1 2 3 2 0 3 0 0]
[2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 0, 2, 2, 1, 2, 1, 0, 1, 0, 1, 0, 2, 2, 0, 3, 1, 1, 2, 2, 0, 3, 0, 1, 3, 0, 2, 1, 0, 3, 2, 0, 2, 3, 2, 3, 0, 3, 2, 1, 0, 1, 0, 0, 3, 0, 3, 0, 3, 3, 2, 3, 0, 2, 3, 0, 1, 3, 1, 2, 2, 0, 3, 1, 0, 2, 3, 0, 2, 1, 2, 3, 0, 1, 3, 0, 1, 0, 1, 2, 2, 2, 3, 0, 0, 1, 1, 2, 0, 2, 3, 3, 0, 3, 1, 0, 0, 3, 3, 3, 2, 2, 0, 1, 2, 2, 2, 2, 3, 1, 2, 3, 1, 2, 2, 0, 1, 3, 0, 0, 2, 3, 1, 2, 3, 3, 3, 1, 3, 2, 1, 3, 3, 2, 1, 2,